In [1]:
import numpy as np
import pandas as pd
import matplotlib as plot
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from scipy.stats import shapiro
import statsmodels.api as sm
import statsmodels.formula.api as smf
import pylab
from sklearn.preprocessing import scale 
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import roc_auc_score,roc_curve

In [2]:
data= pd.read_csv("hmelq.csv")

In [3]:
data.head()

,bad,loan,mortdue,value,reason,job,yoj,derog,delinq,clage,ninq,clno,debtinc
0,0,81200,18834.0,108355.0,HomeImp,NaN,28.0,0.0,0.0,139.14,0.0,14.0,34.042
1,0,12600,103960.0,127384.0,DebtCon,NaN,2.0,0.0,0.0,129.02,0.0,25.0,34.479
2,0,18000,46865.0,61266.0,DebtCon,NaN,5.0,0.0,0.0,102.59,2.0,9.0,26.354
3,0,10300,57676.0,71027.0,DebtCon,NaN,19.0,0.0,0.0,157.52,1.0,11.0,33.992
4,0,9400,56508.0,78358.0,DebtCon,NaN,17.0,0.0,0.0,141.93,0.0,11.0,32.327


In [4]:
data.describe()

,bad,loan,mortdue,value,yoj,derog,delinq,clage,ninq,clno,debtinc
count,3576.000000,3576.000000,3262.000000,3512.000000,3264.000000,3149.000000,3225.000000,3397.000000,3273.000000,3443.000000,2809.000000
mean,0.199385,18529.697987,74013.907992,101076.922759,8.832644,0.261670,0.432558,179.832529,1.186373,21.507116,33.831737
std,0.399594,11262.834060,44016.833772,54865.275351,7.568046,0.865178,1.099230,87.066198,1.728723,10.167490,9.061837
min,0.000000,1100.000000,2619.000000,8800.000000,0.000000,0.000000,0.000000,0.490000,0.000000,0.000000,0.720000
25%,0.000000,11000.000000,46700.250000,65753.000000,3.000000,0.000000,0.000000,115.600000,0.000000,15.000000,29.140000
50%,0.000000,16200.000000,65858.000000,89183.000000,7.000000,0.000000,0.000000,173.640000,1.000000,20.000000,34.747000
75%,0.000000,23100.000000,91717.000000,118928.250000,13.000000,0.000000,0.000000,230.710000,2.000000,26.000000,38.940000
max,1.000000,89900.000000,399550.000000,850000.000000,41.000000,10.000000,15.000000,1168.230000,17.000000,71.000000,203.312000


In [5]:
df=data.copy()

In [6]:
df=df.dropna()
df.head()

,bad,loan,mortdue,value,reason,job,yoj,derog,delinq,clage,ninq,clno,debtinc
153,0,18200,94727.0,136877.0,DebtCon,Mgr,15.0,0.0,0.0,168.96,2.0,26.0,36.056
154,0,21700,79240.0,96784.0,DebtCon,Mgr,5.0,0.0,0.0,64.51,6.0,24.0,38.079
155,0,34100,241931.0,36486.0,DebtCon,Mgr,1.0,0.0,2.0,196.01,3.0,50.0,42.459
156,0,8400,62989.0,76718.0,HomeImp,Mgr,3.0,0.0,2.0,131.47,0.0,22.0,29.200
157,0,17400,25859.0,43684.0,DebtCon,Mgr,16.0,1.0,0.0,95.36,1.0,17.0,27.108


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2018 entries, 153 to 3549
Data columns (total 13 columns):
bad        2018 non-null int64
loan       2018 non-null int64
mortdue    2018 non-null float64
value      2018 non-null float64
reason     2018 non-null object
job        2018 non-null object
yoj        2018 non-null float64
derog      2018 non-null float64
delinq     2018 non-null float64
clage      2018 non-null float64
ninq       2018 non-null float64
clno       2018 non-null float64
debtinc    2018 non-null float64
dtypes: float64(9), int64(2), object(2)
memory usage: 220.7+ KB


In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3576 entries, 0 to 3575
Data columns (total 13 columns):
bad        3576 non-null int64
loan       3576 non-null int64
mortdue    3262 non-null float64
value      3512 non-null float64
reason     3429 non-null object
job        3409 non-null object
yoj        3264 non-null float64
derog      3149 non-null float64
delinq     3225 non-null float64
clage      3397 non-null float64
ninq       3273 non-null float64
clno       3443 non-null float64
debtinc    2809 non-null float64
dtypes: float64(9), int64(2), object(2)
memory usage: 363.3+ KB


In [9]:
dm=pd.get_dummies(df[['reason','job']])
dm.head()

,reason_DebtCon,reason_HomeImp,job_Mgr,job_Office,job_Other,job_ProfEx,job_Sales,job_Self
153,1,0,1,0,0,0,0,0
154,1,0,1,0,0,0,0,0
155,1,0,1,0,0,0,0,0
156,0,1,1,0,0,0,0,0
157,1,0,1,0,0,0,0,0


In [10]:

y=df['bad']


In [11]:
x_=df.drop(['reason','job','bad'],axis=1).astype('float64')

In [12]:
x_.tail()

,loan,mortdue,value,yoj,derog,delinq,clage,ninq,clno,debtinc
3544,7200.0,15637.0,39660.0,6.0,0.0,1.0,354.24,3.0,42.0,33.624
3546,53400.0,228236.0,305514.0,6.0,0.0,0.0,11.15,0.0,2.0,34.558
3547,27300.0,106221.0,137727.0,3.0,0.0,0.0,300.57,3.0,45.0,32.560
3548,37300.0,68789.0,86172.0,2.0,0.0,6.0,235.64,2.0,25.0,114.051
3549,13300.0,25698.0,58760.0,15.0,0.0,0.0,147.91,1.0,10.0,30.962


In [13]:
X=pd.concat([x_,dm],axis=1)

X.head()

,loan,mortdue,value,yoj,derog,delinq,clage,ninq,clno,debtinc,reason_DebtCon,reason_HomeImp,job_Mgr,job_Office,job_Other,job_ProfEx,job_Sales,job_Self
153,18200.0,94727.0,136877.0,15.0,0.0,0.0,168.96,2.0,26.0,36.056,1,0,1,0,0,0,0,0
154,21700.0,79240.0,96784.0,5.0,0.0,0.0,64.51,6.0,24.0,38.079,1,0,1,0,0,0,0,0
155,34100.0,241931.0,36486.0,1.0,0.0,2.0,196.01,3.0,50.0,42.459,1,0,1,0,0,0,0,0
156,8400.0,62989.0,76718.0,3.0,0.0,2.0,131.47,0.0,22.0,29.200,0,1,1,0,0,0,0,0
157,17400.0,25859.0,43684.0,16.0,1.0,0.0,95.36,1.0,17.0,27.108,1,0,1,0,0,0,0,0


In [14]:
#stats model kütüphanesini kullanarak kurulan model
#modele göre loan, mortdue,value,loj ve dummy e dönüştürdüğümüz değişkenler model için anlamsız çıktı.

In [15]:
loj=sm.Logit(y,X)
loj_model=loj.fit()
loj_model.summary()

Optimization terminated successfully.
         Current function value: 0.228362
         Iterations 8


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                    bad   No. Observations:                 2018
Model:                          Logit   Df Residuals:                     2001
Method:                           MLE   Df Model:                           16
Date:                Thu, 06 Aug 2020   Pseudo R-squ.:                  0.2346
Time:                        16:07:37   Log-Likelihood:                -460.84
converged:                       True   LL-Null:                       -602.11
Covariance Type:            nonrobust   LLR p-value:                 1.039e-50
==================================================================================
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
loan            -1.73e-05   1.05e-05     -1.640      0.101    -3.8e-05    3.37e-06
mortdue        -4.777e-06   4.76e-06     -1.004      0.315   -1.41e-05    4.55e-06
value           4.089e-06   4.06e-06      1.006      0.314   -3.87e-06    1.21e-05
yoj               -0.0109      0.013     -0.854      0.393      -0.036       0.014
derog              0.5708      0.130      4.382      0.000       0.315       0.826
delinq             0.7985      0.095      8.448      0.000       0.613       0.984
clage             -0.0046      0.001     -3.354      0.001      -0.007      -0.002
ninq               0.1119      0.049      2.303      0.021       0.017       0.207
clno              -0.0286      0.011     -2.627      0.009      -0.050      -0.007
debtinc            0.1143      0.014      8.222      0.000       0.087       0.142
reason_DebtCon    -3.7938   1.26e+07  -3.02e-07      1.000   -2.46e+07    2.46e+07
reason_HomeImp    -3.8593   1.26e+07  -3.07e-07      1.000   -2.46e+07    2.46e+07
job_Mgr           -1.4722   1.26e+07  -1.17e-07      1.000   -2.46e+07    2.46e+07
job_Office        -2.0716   1.26e+07  -1.65e-07      1.000   -2.46e+07    2.46e+07
job_Other         -1.5260   1.26e+07  -1.22e-07      1.000   -2.46e+07    2.46e+07
job_ProfEx        -1.5377   1.26e+07  -1.22e-07      1.000   -2.46e+07    2.46e+07
job_Sales         -0.2487   1.26e+07  -1.98e-08      1.000   -2.46e+07    2.46e+07
job_Self          -0.7970   1.26e+07  -6.35e-08      1.000   -2.46e+07    2.46e+07
==================================================================================
"""

from sklearn.linear_model import LogisticRegression
Kütüphanesi kullanarak model kuruldu

In [132]:
logistic_regression= LogisticRegression()
model=logistic_regression.fit(X,y)

C:\Users\karde\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [133]:
model.intercept_

array([-0.0086955])

In [134]:
model.coef_

array([[-2.83163861e-05, -4.89389252e-06, -1.61418650e-06,
        -3.31738339e-02,  2.01031589e-02,  3.64166548e-02,
        -7.28657597e-03,  1.07646587e-02, -2.92930551e-02,
         2.39167894e-02, -5.62310945e-03, -3.07239212e-03,
        -5.37405428e-05, -4.14770080e-03, -5.33142381e-03,
        -1.64484468e-03,  1.14343216e-03,  1.33877610e-03]])

In [135]:
y_pred=loj_model.predict(X)

In [136]:
accuracy_score(y,y_pred)
#kurduğumuz modelde verileri test ve train olarak ayırmadan modelimiz %90 öğrendi

0.9117938553022795

In [97]:
confusion_matrix(y,y_pred)

array([[1835,    5],
       [ 178,    0]], dtype=int64)

In [137]:
print(classification_report(y,y_pred))


              precision    recall  f1-score   support

           0       0.91      1.00      0.95      1840
           1       0.00      0.00      0.00       178

    accuracy                           0.91      2018
   macro avg       0.46      0.50      0.48      2018
weighted avg       0.83      0.91      0.87      2018



C:\Users\karde\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [138]:
model.predict(X)[0:10] #kurulan modelde tahmin edilen değerler

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)

In [139]:
y_probs= model.predict_proba(X)
y_probs=y_probs[:,1]

In [140]:
y_pred=[0 if i >0.2 else 1 for i in y_probs ]

In [141]:
y_pred =model.predict(X)


In [142]:
print(classification_report(y,y_pred))

              precision    recall  f1-score   support

           0       0.91      1.00      0.95      1840
           1       0.00      0.00      0.00       178

    accuracy                           0.91      2018
   macro avg       0.46      0.50      0.48      2018
weighted avg       0.83      0.91      0.87      2018



In [110]:
accuracy_score(y,y_pred)

0.9093161546085233

In [111]:
logit_roc_auc= roc_auc_score(y,loj_model.predict(X)) #alanı verir

fpr, tpr, thresholds = roc_curve(y, loj_model.predict_proba(X)[:,1])
#roc curve verir
plt.figure()
plt.plot(fpr, tpr, label='AUC (area = %0.2f)' % logit_roc_auc)
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Oranı')
plt.ylabel('True Positive Oranı')
plt.title('ROC')
plt.show()

AttributeError: 'LogitResults' object has no attribute 'predict_proba'


Modeli test ve train olara ayıralım


In [52]:
x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.20,random_state=42)
#verinin %20sini test olarak  %80 ini modeli öğretmek için kullandık

In [53]:
loj=LogisticRegression(solver='liblinear')
loj_model=loj.fit(x_train,y_train)
loj_model

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

In [54]:
y_pred=model.predict(x_test)

In [62]:
y_probs= model.predict_proba(x_test)
y_probs=y_probs[:,1]

In [110]:
y_pred=[1 if i >0.21 else 0 for i in y_probs ]

In [111]:
accuracy_score(y_test,model.predict(x_test))
#test verileriyle model %91 olarak modeli öğrenmiş

0.9133663366336634

In [112]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.93      0.93      0.93       369
           1       0.31      0.31      0.31        35

    accuracy                           0.88       404
   macro avg       0.62      0.62      0.62       404
weighted avg       0.88      0.88      0.88       404



In [27]:
cross_val_score(model,x_test,y_test,cv=15).mean()
#model açıklanırlığı rasgele mi gelmiş bakmak için
#rastgele 15 örneklemden modeli test ettik model açıklanılırlığı yine yüksek düzeyde

C:\Users\karde\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\karde\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\karde\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\karde\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\karde\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence t

0.9159611992945327

SOLVER='newton-cg' için model kuralım

In [113]:
x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.20,random_state=42)

In [122]:
loj=LogisticRegression(solver='sag')
loj_model=loj.fit(x_train,y_train)
loj_model

C:\Users\karde\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='sag', tol=0.0001, verbose=0,
                   warm_start=False)

In [127]:
y_pred=model.predict(x_train)

In [129]:
y_probs= model.predict_proba(x_train)
y_probs=y_probs[:,1]

In [130]:
y_pred=[1 if i >0.21 else 0 for i in y_probs ]

In [131]:
print(classification_report(y_train,y_pred))

              precision    recall  f1-score   support

           0       0.94      0.91      0.92      1471
           1       0.28      0.35      0.31       143

    accuracy                           0.86      1614
   macro avg       0.61      0.63      0.62      1614
weighted avg       0.88      0.86      0.87      1614

